In [1]:
from platform import python_version
import pandas as pd
import numpy as np
from skbio.stats.composition import clr
import skbio
import re

In [3]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
print('The python version is {}.'.format(python_version()))
print('The pandas version is {}.'.format(pd.__version__))
print('The numpy version is {}.'.format(np.__version__))
print('The skbio version is {}.'.format(skbio.__version__))

The python version is 3.11.12.
The pandas version is 2.1.3.
The numpy version is 1.26.4.
The skbio version is 0.6.2.


loading microbiome abundance table

In [5]:
genus=pd.read_csv( "genus.csv",index_col=0)
genus

,Lysinibacillus,Fusobacterium,Bacteroides,Prevotella,Sphingomonas,Blautia,Faecalibacterium,[Eubacterium]_coprostanoligenes_group,Akkermansia,Lachnoclostridium,...,Acidibacter,Fimbriiglobus,Papillibacter,Pantalinema,Jonquetella,Propionibacterium,Devosia,FTLpost3,Achromobacter,Blastomonas
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR475500,232186,0,2651,11,8991,736,242,3262,52978,961,...,0,0,0,0,0,0,0,0,0,0
Cancer7-2537,0,214716,13442,16,0,23077,37,64,533,2908,...,0,0,0,0,0,0,0,0,0,0
ERR475501,42,0,180089,25,36718,4312,1124,16,0,70389,...,0,0,0,0,0,0,0,0,0,0
ERR475507,0,0,65835,151518,16385,4057,52951,1353,6781,1447,...,0,0,0,0,0,0,0,0,0,0
ERR475506,0,0,1024,164,144878,129257,340,4787,0,173,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR2143774,0,0,528,0,0,999,202,13,837,0,...,0,0,0,0,0,0,0,0,0,0
SRR2143541,0,0,1257,0,0,495,141,122,0,0,...,0,0,0,0,0,0,0,0,0,0
SRR2143550,0,0,1253,1059,0,240,238,73,0,0,...,0,0,0,0,0,0,0,0,0,0


filtering of taxa with low prevalence:

In [7]:
def drop_almost_zero(df, percentage):


    column_cut_off = int(percentage/100*len(df)) 
    b = (df == 0).sum(axis='rows')
    df = df[ b[ b <= column_cut_off].index.values ]
    

    return df

In [9]:
genus_filtered=drop_almost_zero(genus,90)
genus_filtered

,Fusobacterium,Bacteroides,Prevotella,Sphingomonas,Blautia,Faecalibacterium,[Eubacterium]_coprostanoligenes_group,Akkermansia,Lachnoclostridium,Escherichia-Shigella,...,Merdibacter,Adlercreutzia,Shuttleworthia,Anaerofustis,Atopobium,Holdemania,Methylobacterium-Methylorubrum,Gordonibacter,Candidatus_Soleaferrea,GCA-900066755
SampleID,,,,,,,,,,,,,,,,,,,,,
ERR475500,0,2651,11,8991,736,242,3262,52978,961,27326,...,107,0,25,6,5,0,6,0,0,0
Cancer7-2537,214716,13442,16,0,23077,37,64,533,2908,49077,...,0,0,0,0,0,30,0,0,0,0
ERR475501,0,180089,25,36718,4312,1124,16,0,70389,15959,...,0,0,0,0,0,0,101,0,0,103
ERR475507,0,65835,151518,16385,4057,52951,1353,6781,1447,2172,...,6,0,0,0,5,46,0,0,0,0
ERR475506,0,1024,164,144878,129257,340,4787,0,173,19,...,131,0,0,6,0,0,30,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR2143774,0,528,0,0,999,202,13,837,0,0,...,0,0,0,0,0,0,0,0,0,0
SRR2143541,0,1257,0,0,495,141,122,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SRR2143550,0,1253,1059,0,240,238,73,0,0,10,...,0,0,0,0,0,0,0,0,0,0


loading metadata information:

In [11]:
metadata=pd.read_csv( "metadata.csv",index_col=0)
metadata

,SubjectID,Diagnosis,Study,Country,Age,Gender,BMI,Total_reads,Reads_after_QC,used_read_count_percentage,16S_Region,Seq_platform
SampleID,,,,,,,,,,,,
Adenoma1-2065,Adenoma1-2065,Adenoma,Zackular,CA,76,m,23,139559,107534,77.05,V4,Illumina MiSeq
Adenoma10-2757,Adenoma10-2757,Adenoma,Zackular,CA,37,m,24,69882,53194,76.12,V4,Illumina MiSeq
Adenoma11-2775,Adenoma11-2775,Adenoma,Zackular,CA,65,f,21,116629,91060,78.08,V4,Illumina MiSeq
Adenoma12-2799,Adenoma12-2799,Adenoma,Zackular,CA,82,m,36,71532,51092,71.43,V4,Illumina MiSeq
Adenoma13-2803,Adenoma13-2803,Adenoma,Zackular,CA,62,m,28,146499,111419,76.05,V4,Illumina MiSeq
...,...,...,...,...,...,...,...,...,...,...,...,...
SRR2144126,3535650,Adenoma,Baxter,USA,53,f,34,41261,31732,76.91,V4,Illumina MiSeq
SRR2144127,3537650,Normal,Baxter,USA,75,m,27,65890,54475,82.68,V4,Illumina MiSeq
SRR2144128,3551650,Adenoma,Baxter,USA,56,f,27,48961,39779,81.25,V4,Illumina MiSeq


centered log-ratio transformation (clr):

In [13]:
X_arr=np.array(genus_filtered)
for i in range(X_arr.shape[0]):
    
    X_arr[i][X_arr[i]==0]=1

In [15]:
X_temp=pd.DataFrame(X_arr,index=genus_filtered.index,columns=genus_filtered.columns)
X_clr=pd.DataFrame(clr(X_temp),index=genus_filtered.index,columns=genus_filtered.columns)

In [17]:
#changing columns names
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X_clr.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_clr.columns.values]

preparing the dataset for the machine learning classification (adding Age, gender, BMI, Country)

In [19]:
for i in metadata.index:
    if metadata.loc[i,'Gender'].startswith("f"):
        metadata.at[i,'Gender']='f'
    elif metadata.loc[i,'Gender'].startswith("m"):
        metadata.at[i,'Gender']='m'

In [21]:
for i in metadata.index:
    if metadata.loc[i,'Gender'].startswith("F"):
        metadata.at[i,'Gender']='f'
    elif metadata.loc[i,'Gender'].startswith("M"):
        metadata.at[i,'Gender']='m'

In [23]:
for i in metadata.index:
    if metadata.loc[i,'BMI']=='#¡DIV/0!':
        metadata.at[i,'BMI']=np.nan

In [25]:
metadata['Age']=metadata['Age'].astype(int)
metadata['BMI']=metadata['BMI'].astype(float)

In [27]:
X_clr=X_clr.loc[metadata.index]

In [29]:
#adding BMI, age, gender
X_clr['BMI']=metadata.loc[X_clr.index]['BMI']
X_clr['Age']=metadata.loc[X_clr.index]['Age']
X_clr['Gender']=metadata.loc[X_clr.index]['Gender']

In [31]:
#one hot encoding
X_clr = pd.get_dummies(X_clr, columns = ['Gender'])
X_clr.drop(['Gender_f'],axis=1,inplace=True)

In [33]:
X_clr['Country']=metadata.loc[X_clr.index]['Country']

In [35]:
#one hot encoding
X_clr = pd.get_dummies(X_clr, columns = ['Country'])

In [37]:
#adding diagnosis
X_clr['Diagnosis']=metadata.loc[X_clr.index]['Diagnosis']

In [23]:
X_clr.to_csv('X_clr.csv')

In [39]:
np.unique(X_clr['Diagnosis'],return_counts=True)

(array(['Adenoma', 'Cancer', 'Large adenoma', 'Normal', 'Small adenoma'],
       dtype=object),
 array([228, 191,  13, 251,  25]))

In [41]:
#selecting only cancer and adenoma subjects
X_clr_adenoma=X_clr.loc[(X_clr['Diagnosis']=='Adenoma')]
X_clr_small_adenoma=X_clr.loc[(X_clr['Diagnosis']=='Small adenoma')]
X_clr_large_adenoma=X_clr.loc[(X_clr['Diagnosis']=='Large adenoma')]
X_clr_cancer=X_clr.loc[X_clr['Diagnosis']=='Cancer']

final dataset

In [44]:
df=pd.concat([X_clr_adenoma,X_clr_small_adenoma,X_clr_large_adenoma,X_clr_cancer]).sample(frac=1,random_state=5)
df

,Fusobacterium,Bacteroides,Prevotella,Sphingomonas,Blautia,Faecalibacterium,_Eubacterium__coprostanoligenes_group,Akkermansia,Lachnoclostridium,Escherichia-Shigella,...,Gordonibacter,Candidatus_Soleaferrea,GCA-900066755,BMI,Age,Gender_m,Country_CA,Country_FRA,Country_USA,Diagnosis
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR2143615,-1.486105,4.687681,-1.486105,-1.486105,7.144060,5.064976,4.123367,4.946835,-1.486105,1.286484,...,1.649390,-1.486105,-1.486105,24.0,67,False,False,False,True,Adenoma
SRR2143547,-2.345212,5.676373,-2.345212,-2.345212,6.429720,3.944504,3.737007,6.546987,-2.345212,0.950625,...,-2.345212,-2.345212,-2.345212,30.0,69,False,False,False,True,Adenoma
Cancer24-2727,-1.052114,6.289693,-0.541289,-2.843874,6.363462,5.486025,3.215249,1.345781,1.330513,2.205982,...,-0.071285,-1.457580,-2.843874,33.0,48,True,False,False,True,Cancer
ERR475530,-2.683090,7.205182,4.905740,4.228657,5.665685,2.906030,4.494311,4.199348,4.955349,2.320856,...,-3.376237,-3.376237,-0.543024,23.0,78,False,False,True,False,Large adenoma
Cancer21-2671,5.535763,8.052574,-2.234882,-2.234882,2.390091,1.549307,3.237389,6.582712,4.900805,3.298507,...,-2.234882,-2.234882,-2.234882,33.0,60,True,False,False,True,Cancer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR2143784,-1.595084,6.719013,-1.595084,-1.595084,5.787040,5.172259,6.615040,1.806113,4.847456,0.969865,...,-1.595084,-1.595084,-1.595084,31.0,78,True,False,False,True,Cancer
SRR2143752,-2.897568,6.511377,3.453318,-2.897568,5.657114,5.565802,3.938691,1.034258,2.278582,3.209455,...,-1.105808,-2.897568,-2.897568,31.0,69,True,False,False,True,Adenoma
SRR2143952,-1.374877,6.215470,-1.374877,-1.374877,6.347358,6.046899,3.976981,2.556949,2.516944,2.955857,...,-1.374877,-1.374877,-1.374877,23.0,81,False,False,False,True,Adenoma


In [46]:
df.to_csv('df.csv')